In [1]:
import numpy as np
import sklearn_crfsuite
import pandas as pd
import nltk
import pickle
from rdflib import Graph, URIRef
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import csv
import numpy as np
import os
import rdflib
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import json
import time
import atexit
import getpass
import requests  # install the package via "pip install requests"
from collections import defaultdict
import pandas as pd
from rdflib import Graph
import rdflib
import numpy as np
import csv
from sklearn.metrics import pairwise_distances

#from sparknlp.annotator import *
#from sparknlp.base import *
#import sparknlp
#import spark
import urllib.request

#from sparknlp.annotator import *
#from sparknlp.base import *
#import sparknlp
#spark = sparknlp.start()

#from sparknlp.pretrained import PretrainedPipeline

###
#import spacy

## Answer factual questions
Model training need to be applied only once. The function is at the bottom

In [2]:
class NerTrainer:
    #Most of the funcitons are from the Graphical Models Tutorial from the class

    def __init__(self):
        self.x_train = pd.read_csv('engtrain_clean.csv', encoding="ISO-8859-1")
        self.x_test = pd.read_csv('engtest_clean.csv', encoding="ISO-8859-1")
        print("Loaded data & initialized NER-model")

    def collate(self, dataframe):
        agg_func = lambda s: [(w, pos, t) for w, pos, t in
                              zip(s['WORD'].values.tolist(), s['POS'].values.tolist(), s['ï»¿TAG'].values.tolist())]
        grouped = dataframe.groupby('SENTENCE').apply(agg_func)
        return list(grouped)

    # counts sentences in given input data
    def count_sentences(self, data):
        counter = 0
        for index, row in data.iterrows():
            data.at[index, 'SENTENCE'] = counter
            if pd.isna(data.iloc[index, 0]):
                counter += 1
        data["POS"] = "0"
        data = data.dropna()
        data = data.reset_index(drop=True)
        return data

    def word2features(self, sent, i):
        word = sent[i][0]
        postag = sent[i][1]

        features = {
            'word.lower()': word.lower(),  # the word in lowercase
            'word[-3:]': word[-3:],  # last three characters
            'word[-2:]': word[-2:],  # last two characters
            'word.isupper()': word.isupper(),  # true, if the word is in uppercase
            'word.istitle()': word.istitle(),
            # true, if the first character is in uppercase and remaining characters are in lowercase
            'word.isdigit()': word.isdigit(),  # true, if all characters are digits
            'postag': postag,  # POS tag
            'postag[:2]': postag[:2],  # IOB prefix
        }

        if i > 0:
            word1 = sent[i - 1][0]  # the previous word
            postag1 = sent[i - 1][1]  # POS tag of the previous word
            features.update({
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
                '-1:postag[:2]': postag1[:2],
            })  # add some features of the previous word
        else:
            features['BOS'] = True  # BOS: begining of the sentence

        if i < len(sent) - 1:
            word1 = sent[i + 1][0]  # the next word
            postag1 = sent[i + 1][1]  # POS tag of the next word
            features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
            })  # add some features of the next word
        else:
            features['EOS'] = True  # EOS: end of the sentence
        return features

    def sent2features(self, sent):
        return [self.word2features(sent, i) for i in range(len(sent))]

    def sent2labels(self, sent):
        return [label for _, _, label in sent]

    def tag_data(self, data_to_tag):
        tags = []
        x_train = data_to_tag.groupby(['SENTENCE'])
        x_train = x_train.apply(np.mean)

        # x_train.shape[0] will give us the number of sentences to loop through and tag them using nltk lib
        # first we locate the sentence in our input data, then we hand the sentence as a list of words to the nltk function which tags the words.
        for i in range(x_train.shape[0]):
            sentence = data_to_tag.loc[data_to_tag["SENTENCE"] == i]
            sentence = sentence['WORD'].tolist()
            tag = nltk.pos_tag(sentence)
            tags.append(tag)

        #now we add the found tags to the actual data in the data set
        counter = 0
        for sentence in tags:
            for word in sentence:
                data_to_tag.at[counter, 'POS'] = word[1]
                counter += 1
        print("data to tag in ""tag_data()"":" )
        print(data_to_tag)
        print("#######################################################")
        return data_to_tag

    #code copied from graphical_model tutorial and collected under one train function
    def train(self):
        self.x_train = self.count_sentences(self.x_train)
        self.x_test = self.count_sentences(self.x_test)

        self.x_train = self.tag_data(self.x_train)
        self.x_test = self.tag_data((self.x_test))

        self.x_train = self.collate(self.x_train)
        self.x_test = self.collate((self.x_test))

        X_train, y_train = [self.sent2features(s) for s in self.x_train], [self.sent2labels(s) for s in self.x_train]
        X_test, y_test = [self.sent2features(s) for s in self.x_test], [self.sent2labels(s) for s in self.x_test]

        crf = sklearn_crfsuite.CRF(
            algorithm='l2sgd',  # l2sgd: Stochastic Gradient Descent with L2 regularization term
            max_iterations=1000,  # maximum number of iterations
        )
        crf_model = crf.fit(X_train, y_train)
        return  crf_model

    #predict tags of given input sentence
    def predict_tags(self, model, sentence):
        text = []

        #create list of workds from input sentence
        words = word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        text.append(tagged_words)

        empty_list = word_tokenize("")
        empty_tagged_words = nltk.pos_tag(empty_list)
        text.append(empty_tagged_words)

        X_train = [self.sent2features(s) for s in text]
        y = model.predict(X_train)

        #remove any punctuation mark
        if sentence[0][-1][0] in ['?', '!', '¨.']:
            y[0][-1] = '0'

        #Create output dataframe with two new columns and the predicted tag
        tagged_words = np.asarray(tagged_words)
        df_res = pd.DataFrame(tagged_words, columns=['WORD', 'POS'])
        df_res['TAG'] = y[0]
        return df_res

In [3]:
# Class to identify the input question. Question content and type.
class QuestionAnalyser:
    def __init__(self):
        print("Initialized Question Analyser")

    #Funciton to identify if question is about the director ot the actor.
    #Counts nouns and verbs that are related to either. Role with highest score is designated as question content.
    def identity_counter(self, lemma_verbs, lemma_nouns):

        lemma_words = lemma_verbs + lemma_nouns

        candidates = {"director" : ['director', "filmmaker",'direct', 'produce', 'filmed', 'lead'],
                      "actor" : ['actor', 'role', 'star', 'act', 'play', 'star'],
                      "screenwriter":  ['screenwriter', 'scriptwriter', 'author', 'screenwrite', 'write', 'script'],
                      "MPA film rating" : ['mpaa', 'rating', 'ranking', 'grade', 'rate', 'rank', 'grade'],
                      "image" : ['image', 'picture', 'face', 'look', "show"],
                      "genre" : ['genre', 'type', 'style', 'category'],
                      "box office" : ["box", "office", "ticket", "booth"],
                      "publication date": ["publication", "publish", "release"],
                      "cast member": ["cast", "casting"],
                      "producer": ["producer", "produce", "maker", "make", "production"],
                      "executive producer": ["producer", "produce", "executive", "maker", "make"],
                      "based on": ["base", "based"],
                      "country of origin" : ["origin"],
                      "filming location": ["location", "place", "locate", "film", "shoot"],
                      "narrative location": ["location", "place", "locate", "narrate", "narrative"],
                      "director of photography": ["photography"],
                      "Box Office Mojo film ID": ["box", "office", "ticket", "booth", "Mojo", "ID", "id"],
                      "set in period": ["period"],
                      "duration": ["duration"],
                      "film editor": ["editor", "edit", "compiler", "compile"],
                      "FilmAffinity ID": ["FilmAffinity", "filmaffinity"],
                      "Filmportal": ["Filmportal", "filmportal"],
                      "main subject": ["subject", "topic"],
                      "composer": ["composer", "compose"],
                      "distributed by": ["distributer", "distribute", "trader", "trade", "seller"],
                      "costume designer": ["costume", "outfit", "wardrobe", "dress"],
                      "production designer": ["producer", "produce", "executive", "maker", "make", "designer", "design"],
                      "Kinopoisk film ID": ["Kinopoisk", "kinopoisk"],
                      "average shot length": ["average", "shot", "length"],
                      "exploitation visa number": ["visa", "explotation"],
                      "original film format": ["original", "film", "format"],
                      "film script": ["script", "scripte"],
                      "title": ["title"],
                      "original language of film or TV show": ["original", "language", "initial"],
                      "award received": ["award"],
                      "media franchise": ["franchise"],
                      "part of the series": ["series"],
                      "budget": ["budget", "cost"]
                      }


        counter = {"director": 0, "actor": 0, "screenwriter": 0, "MPA film rating": 0, "image": 0, "genre": 0,
                   "box office": 0, "publication date": 0, "cast member": 0, "producer": 0, "executive producer":0,
                   "based on": 0, "country of origin": 0, "filming location": 0, "narrative location": 0, "director of photography": 0,
                   "Box Office Mojo film ID":0, "set in period": 0, "duration": 0, "film editor": 0, "FilmAffinity ID": 0,
                   "Filmportal ID": 0, "main subject": 0, "composer": 0, "distributed by": 0,  "costume designer": 0,
                   "production designer": 0, "Kinopoisk film ID": 0, "exploitation visa number": 0, "average shot length": 0,
                   "original film format": 0, "film script": 0, "title": 0, "award": 0, "media franchise": 0, "part of the series": 0,
                   "budget": 0
                   }

        #check occurences of words in input sentence
        for key in candidates.keys():
            for w in candidates[key]:
                if w in lemma_words:
                    counter[key] += 1

        max_value = max(counter, key=counter.get)

        #double check for box office and avg shot length since it should appear in word combinations
        if max_value == "Box Office Mojo film ID":
            lemma_lower = [w.lower() for w in lemma_words]
            if "id" not in lemma_lower:
                max_value = "box office"

        if max_value == "average shot length":
            counter["average shot length"] -= 1
            max_value = max(counter, key=counter.get)
        if max_value == "original film format":
            counter["original film format"] -= 1
            max_value = max(counter, key=counter.get)


        #decide between location properties:
        if max_value in ["country of origin", "filming location", "narrative location"]:
            if "origin" in lemma_verbs:
                max_value = "country of origin"
            elif "film" or "shoot" in lemma_verbs:
                max_value = "filming location"
            else:
                max_value = "narrative location"

        #decide between the production rolees:
        if max_value in ["production designer", "executive producer"]:
            if "designer" in lemma_words:
                max_value = "production designer"
            elif  "design" in lemma_words:
                max_value = "production designer"
            elif "executive" in lemma_words:
                max_value = "executive producer"
            else: max_value = "producer"


        return max_value



    def identify_predicate(self, question):

        # Get untaggde words
        untagged_words = question[question["TAG"]=='0']
        verbs = untagged_words[untagged_words['POS'].isin(['VP', 'VBD', 'VBG', 'VBP', 'VBZ'])]['WORD']
        nouns = untagged_words[untagged_words['POS'].isin(['NN', 'NNP', 'NNPS', "JJ"])]['WORD']

        #Lemmatize means to get the root of the word
        lemma = WordNetLemmatizer()
        lemma_verbs = [lemma.lemmatize(v) for v in verbs]
        lemma_nouns = [lemma.lemmatize(v) for v in nouns]
        print(lemma_nouns)
        print(lemma_verbs)

        ########################################################
        pred = self.identity_counter(lemma_nouns = lemma_nouns, lemma_verbs=lemma_verbs)
        return pred


In [4]:
class TypoCorrector:

    @staticmethod
    def change_minus_sign(input):
        # replace minus with dash
        if "-" in input:
            input = input.replace("-", "–")

        return input

    @staticmethod
    def change_dash_sign(input):
        # replace minus with dash
        if "-" in input:
            input = input.replace("–","-")

        return input

In [5]:
class  ModelTrainer:

    def __init__(self, ner_model, training_data, test_data):
        self.ner_model = ner_model
        self.x_train = pd.read_csv(training_data, encoding="ISO-8859-1")
        self.x_test = pd.read_csv(test_data, encoding="ISO-8859-1")
        print("Loaded data successfully")



    def train_model(self):
        self.ner_model.train(self.x_train,self.x_test)

In [6]:
class Crowdsourcing:

    def __init__(self):
        self.df = pd.read_csv('D:/deanh/Uni/Master/HS22/Advanced AI/Project/crowdsourcing/crowd_data.tsv', sep='\t')

    def clean_crowdsourcingfile(self):
        #remove % sign from approvalrate to handle as integer
        self.df["LifetimeApprovalRate"] = [int(word[:-1]) for word in self.df['LifetimeApprovalRate']]

        # We define upper and lower bounds for response time and approval rate by using mean and standard deviation
        # In believe that workers outside of these bounds are malicious. We add +5% to all bounds to include valid edge cases
        # We also drop rows that indicate as CORRECT but say the know an error and where to handle it

        x = "WorkTimeInSeconds"
        y = "LifetimeApprovalRate"

        mean_approvalRate = self.df[y].mean()
        std_approvalRate = self.df[y].std()

        mean_responseTime = self.df[x].mean()
        std_responseTime = self.df[x].std()

        upperbound_responseTime = mean_responseTime + std_responseTime + 5
        lowerbound_responseTime = mean_responseTime - std_responseTime - 5

        upperbound_approvalRate = mean_approvalRate + std_approvalRate + 5
        lowerbound_approvalRate = mean_approvalRate - std_approvalRate -5

        self.df = self.df.drop(self.df[(self.df.WorkTimeInSeconds > upperbound_responseTime)].index)
        self.df = self.df.drop(self.df[(self.df.WorkTimeInSeconds < lowerbound_responseTime)].index)

        self.df = self.df.drop(self.df[(self.df.LifetimeApprovalRate > upperbound_approvalRate)].index)
        self.df = self.df.drop(self.df[(self.df.LifetimeApprovalRate < lowerbound_approvalRate)].index)

        self.df = self.df.drop(self.df[(self.df['AnswerLabel'] == "CORRECT") & (self.df["FixPosition"] == "yes")].index)

        #There is a type in Input2ID column
        self.df["Input2ID"] = self.df["Input2ID"].replace("wdt:.P344", "wdt:P344")

        return self.df

    def crowdsourcing_answer(self, sub, pred):
        rows = self.df.loc[self.df["Input1ID"] == "wd:"+sub]
        rows = rows.loc[rows["Input2ID"] == "wdt:"+pred]

        agree = rows.loc[rows["AnswerLabel"] == "CORRECT"]
        disagree = rows.loc[rows["AnswerLabel"] == "INCORRECT"]
        value = rows.iloc[0]['Input3ID']

        if agree.shape[0] > disagree.shape[0]:
            return("The crowd agrees that the answer is " + str(value))
        if agree.shape[0] < disagree.shape[0]:
            return("The crowd disagrees that the answer is " + str(value))
        else:
            return("The crowd is uncertain that the answer is " + str(value))

In [7]:
class KGLoader:
    def __init__(self):
        pass

    def load_KG(self):
        print("Loading knowledge graph from source")
        movie_graph = Graph()
        movie_graph.parse(source='datasets/14_graph.nt', format='turtle')
        print("succesfully loaded knowledge graph")
        return movie_graph

    def load_embeddings(self, graph):
        #add path parameter to retrive dictionaries?

        DDIS = rdflib.Namespace('http://ddis.ch/atai/')
        RDFS = rdflib.namespace.RDFS
        SCHEMA = rdflib.Namespace('http://schema.org/')

        # load the embeddings
        entity_emb = np.load(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'entity_embeds.npy'))
        relation_emb = np.load(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'relation_embeds.npy'))

        # load the dictionaries
        with open(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'entity_ids.del'), 'r') as ifile:
            ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
            id2ent = {v: k for k, v in ent2id.items()}
        with open(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'relation_ids.del'), 'r') as ifile:
            rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
            id2rel = {v: k for k, v in rel2id.items()}

        ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
        lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

        embedding = (entity_emb, relation_emb, ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent)

        return embedding

In [8]:
# Returns a SQL statement as a string
class QueryProducer:
    WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
    WD = rdflib.Namespace("http://www.wikidata.org/entity/")

    def __init__(self):
        pass

    def director_query(self, title):
        query ='''
            SELECT ?name
            WHERE {
            ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                  rdfs:label "''' + title + '''"@en ;
                  ns1:P57 ?director .
            ?director rdfs:label ?name
            }
            '''
        return query

    # returns a dict, with all film titles and their entity Id
    def all_films_id_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")

        query ='''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wds: <http://www.wikidata.org/entity/statement/>
        PREFIX wdv: <http://www.wikidata.org/value/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX p: <http://www.wikidata.org/prop/>
        PREFIX ps: <http://www.wikidata.org/prop/statement/>
        PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX bd: <http://www.bigdata.com/rdf#>

        SELECT ?item ?label
            WHERE{
            ?item wdt:P31 <http://www.wikidata.org/entity/Q11424> .
            ?item rdfs:label ?label .
            FILTER(LANG(?label) = "en").
        }
                    '''
        response = moviegraph.query(query)
        id_to_film = {e[len(WD):]: str(i) for e,i in response}
        film_to_id = {value:key for key, value in id_to_film.items()}

        # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}

        return  film_to_id

        # returns a dict, with all anime titles and their entity Id
    def all_animes_id_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")

        query ='''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wds: <http://www.wikidata.org/entity/statement/>
        PREFIX wdv: <http://www.wikidata.org/value/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX p: <http://www.wikidata.org/prop/>
        PREFIX ps: <http://www.wikidata.org/prop/statement/>
        PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX bd: <http://www.bigdata.com/rdf#>

        SELECT ?item ?label
            WHERE{
            ?item wdt:P31 <http://www.wikidata.org/entity/Q20650540> .
            ?item rdfs:label ?label .
            FILTER(LANG(?label) = "en").
        }
                    '''
        response = moviegraph.query(query)
        id_to_film = {e[len(WD):]: str(i) for e,i in response}
        film_to_id = {value:key for key, value in id_to_film.items()}

        # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}

        return  film_to_id

    def all_actors_id_and_IMDb_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        query ='''
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wds: <http://www.wikidata.org/entity/statement/>
            PREFIX wdv: <http://www.wikidata.org/value/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX p: <http://www.wikidata.org/prop/>
            PREFIX ps: <http://www.wikidata.org/prop/statement/>
            PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX bd: <http://www.bigdata.com/rdf#>

            SELECT ?item ?label ?IMDb_ID
                WHERE{
                ?item wdt:P31 wd:Q5 .
                ?item wdt:P106 wd:Q33999 .
                ?item rdfs:label ?label .
                ?item wdt:P345 ?IMDb_ID  .
            }


            '''

        response = moviegraph.query(query)
        ID_to_actor = {e[len(WD):]: str(i) for e,i, IDMb in response}
        actor_to_Id = {value:key for key, value in ID_to_actor.items()}

        ID_to_actor = {e[len(WD):]: str(IDMb) for e,i, IDMb in response}

        return [actor_to_Id, ID_to_actor]

    def predicateID_query(self, proposition):
        query ="""

        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>

        SELECT ?obj WHERE {
            ?obj rdfs:label """ + '"' + proposition + '"' + """@en .

        }
        """
        entity =moviegraph.query(query)
        df = pd.DataFrame(entity.bindings)
        url = str((df.iloc[0][0]))
        id = url[len(QueryProducer.WDT):]

        return id

    def any_property_value_query(self, title, id, propertyname):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        
        if " " in propertyname:
            propertyname = propertyname.replace(" ", "_")

        query ='''
                SELECT ?name
                WHERE {
                ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                      rdfs:label "''' + title + '''"@en ;
                      ns1:'''+id+''' ?'''+propertyname+''' .
                ?'''+propertyname+''' rdfs:label ?name
                }
                '''
        return query

In [9]:
KGloader = KGLoader()
moviegraph = KGloader.load_KG()
embedding = KGloader.load_embeddings(moviegraph)
print("loaded KG and embeddings")


Loading knowledge graph from source
succesfully loaded knowledge graph
loaded KG and embeddings


In [10]:
Q = QueryProducer()
anime_dict = Q.all_animes_id_query()
films_dict = Q.all_films_id_query()
actors_ID_dict = Q.all_actors_id_and_IMDb_query()[0]
actor_IMDb_dict = Q.all_actors_id_and_IMDb_query()[1]
print("loaded film and actor dicts")


loaded film and actor dicts


In [11]:
image_frame = pd.DataFrame.from_dict(json.load(open("images.json")))
print("loaded image file")

loaded image file


In [12]:
C = Crowdsourcing()
clean_crowdsourcingfile = C.clean_crowdsourcingfile()
print("loaded crowdsourcing file")

loaded crowdsourcing file


In [13]:
class LookupHandler:
    def __init__(self, film_ids, actor_ids, actor_IMDb, query_producer, moviegraph, embedding, images, crowdsourcing):
        self.film_ids = film_ids
        self.actor_IDs = actor_ids
        self.actor_IMDb = actor_IMDb
        self.query_producer = query_producer
        self.movie_graph = moviegraph
        self.embedding = embedding
        self.images = images
        self.crowdsourcing = crowdsourcing
    
    def film_id_finder(self, film_title):
        # Film entity. Using lowercase title for lookup
        film_title = film_title.lower()
        try:
            film_id = self.film_ids[film_title.lower()]
            return film_id
        except:
            print("could not find film, trying to find it correcting typos...")
            pass
        try:
            film_title = TypoCorrector.change_minus_sign(film_title)
            film_id = self.film_ids[film_title.lower()]
            return film_id
        except:
            pass
        try:
            film_title = TypoCorrector.change_dash_sign(film_title)
            film_id = self.film_ids[film_title.lower()]
            return film_id
        except:
            print(film_title)
            raise KeyError("Film title contains typos or does not exist!")

        return film_id

    def graph_lookup(self, film_title, pred):
        
        pred_id = self.query_producer.predicateID_query(pred)
        entity = ""
        query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
        entity = self.movie_graph.query(query)
        df_entity = pd.DataFrame(entity.bindings)

        if df_entity.empty:
            film_title = TypoCorrector.change_minus_sign(film_title)
            query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
            entity = self.movie_graph.query(query)
            df_entity = pd.DataFrame(entity.bindings)

            if df_entity.empty:
                film_title = TypoCorrector.change_dash_sign(film_title)
                query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
                entity = self.movie_graph.query(query)
                df_entity = pd.DataFrame(entity.bindings)

        # takes all answers but responds with the first one
        property_value = []
        for _ in entity:
            property_value.append(str(_.name))
        return property_value

    def embedding_lookup(self, film_title, predicate):
        entity_emb, relation_emb, ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent = self.embedding

        # define some prefixes
        WD = rdflib.Namespace('http://www.wikidata.org/entity/')
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        DDIS = rdflib.Namespace('http://ddis.ch/atai/')
        RDFS = rdflib.namespace.RDFS
        SCHEMA = rdflib.Namespace('http://schema.org/')

        film_id = self.film_id_finder(film_title)
        head = entity_emb[ent2id[WD[film_id]]]

        if predicate =='genre':
            # "genre" relation
            pred = relation_emb[rel2id[WDT.P136]]
        if predicate == 'rating':
            # "MPAA film rating" relation
            pred = relation_emb[rel2id[WDT.P1657]]
        if predicate == 'screenwriter':
            # "screenwriter " relation
            pred = relation_emb[rel2id[WDT.P58]]

        # add vectors according to TransE scoring function.
        lhs = head + pred
        # compute distance to *any* entity
        dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
        # find most plausible entities
        most_likely = dist.argsort()
        # compute ranks of entities
        ranks = dist.argsort().argsort()

        top_ten_results = pd.DataFrame([
                (id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], rank+1)
                for rank, idx in enumerate(most_likely[:10])],
                columns=('Entity', 'Label', 'Score', 'Rank'))
        best_result = top_ten_results.iloc[0:2]["Label"]
        best_result = best_result

        return best_result

    def image_lookup(self, name, asFilmIdentifiedName):
        try:
            actor_Id = self.actor_IDs[name]
        except:
            actor_Id = self.actor_IDs[asFilmIdentifiedName]

        IMDb_Id = self.actor_IMDb[actor_Id]
        valid_options = self.images.loc[self.images["cast"].str.contains(IMDb_Id, regex=False)]
    
        #only images where actor appears alone and its a poster. Take a random sample
        valid_options = valid_options.loc[valid_options["type"]=="poster"]
        valid_options = valid_options.query("cast.str.len() == 1", engine="python")
        valid_options = valid_options["img"]
        valid_options = valid_options.sample()

        url = str(valid_options.iloc[0])
        return url

    def crowdsourcing_lookup(self, filmtitle, pred_id):
        film_id = self.film_id_finder(filmtitle)
        answer = self.crowdsourcing.crowdsourcing_answer(film_id, pred_id)
        return answer

In [14]:
#Takes an argument "training" to determine if it should create and save a model or load an existing one
class RunProcess:
    def __init__(self, training, knowledgegraph):
        print("Starting process")
        self.ner_model = NerTrainer()
        if training:
            self.crf_model = self.ner_model.train()
            print("Created and trained model succesfully")
            with open('crf_model.pkl', 'wb') as f:
                pickle.dump(self.crf_model, f)
            print("Saved crf-model")
        else:
            #self.ner_model = NerDLModel.pretrained('ner_mit_movie_simple_distilbert_base_cased', 'en')\
            #.setInputCols(['document', 'token', 'embeddings']).setOutputCol('ner')
            with open('crf_model.pkl', 'rb') as f:
                self.crf_model = pickle.load(f)
            print("loaded ner-mnodel")

        self.question_analyser = QuestionAnalyser()
        self.query_producer = QueryProducer()
        print("Queryproduce initialized")

        self.movie_graph = knowledgegraph
        self.embedding = embedding

        self.anime_ids = anime_dict
        self.normal_film_ids =films_dict
        self.film_ids = {**self.anime_ids, **self.normal_film_ids} #combine both to one dict
        print("created film dict")

        self.crowdourcing = Crowdsourcing()

        self.actor_IDs = actors_ID_dict
        self.actor_IMDb = actor_IMDb_dict

        print("waiting for image file")
        self.images = image_frame

        self.lookup_handler = LookupHandler(self.film_ids, self.actor_IDs, self.actor_IMDb, 
            self.query_producer, self.movie_graph, self.embedding, self.images, self.crowdourcing)

        print("----------Run Initialized!----------")

    #
    def ask_question(self):
        question = input("Please ask your question")
        return question

    def identify_subject_and_predicate(self, question):
        tagged_entities = self.ner_model.predict_tags(self.crf_model, question)
        print("Sentence with tags: ")
        print(tagged_entities)
        print("#################################################################")
        identified_predicate = self.question_analyser.identify_predicate(tagged_entities)

        #TODO Here we can also check for actors. ALSO we can check if I-title is a questions mark, comma etc.

        film_wordlist = tagged_entities['WORD'][tagged_entities['TAG'].isin(['B-TITLE','I-TITLE'])]
        film_title = ''
        for w in film_wordlist:
            film_title = film_title + w + ' '

        actor_wordlist = tagged_entities['WORD'][tagged_entities['TAG'].isin(['B-ACTOR','I-ACTOR'])]
        actor = ''
        for w in actor_wordlist:
            actor = actor + w + ' '
        # need to remove empry string at end of concatenation
        film_title = film_title[:-1]
        actor = actor[:-1]

        print("Asking for: " + identified_predicate + " in: " + film_title + " or actor : " + actor)
        return film_title, actor, identified_predicate


        film_id = self.film_id_finder(filmtitle)
        answer = self.crowdourcing.crowdsourcing_answer(film_id, pred_id)
        return answer

    def answer_movie_question(self):
        question = self.ask_question()

        film_title, actor, pred = self.identify_subject_and_predicate(question)

        pred_id = self.query_producer.predicateID_query(pred)

        print("subject: "+film_title + " | actor: " +actor+ " | predicate: "+pred+" | predicate ID: "+pred_id ) 
        print("##################################################################")

        # To perform the query, we must remove punctuations.

        if len(film_title) >= 1:
            if film_title[-1] in ['?', '!', '.']:
                film_title = film_title[:-2]
        # intermediate punctions have a space in front of them that needs to be removed
        if " :" in film_title:
            film_title = film_title.replace(" :", ":")

        if len(actor) >= 1:
            if actor[-1] in ['?', '!', '.']:
                actor = actor[:-2]

        #########################################################################

        if pred == 'image':
           image_url = self.lookup_handler.image_lookup(name = actor, asFilmIdentifiedName=film_title)
           print("Image of " + film_title + actor + " at this url: "+ image_url)

        else: 
            try:
                answer_crowd = self.lookup_handler.crowdsourcing_lookup(film_title, pred_id)
                print("answer from the crowd " + str(answer_crowd))
            except:
                print("Not found in crowdsourcing")
                pass

            try: 
                answer_graph = self.lookup_handler.graph_lookup(film_title, pred)
                print("answer of graph lookup " + answer_graph[0])
            except:
                print("Not found in graph")
                pass

            try:
                answer_embedding = self.lookup_handler.embedding_lookup(film_title, pred)
                print("embedding answer: ")
                print(answer_embedding)
            except:
                print("Could not find: " + film_title)
                pass

In [15]:
Process3 = RunProcess(training=False, knowledgegraph=moviegraph)

Starting process
Loaded data & initialized NER-model
loaded ner-mnodel
Initialized Question Analyser
Queryproduce initialized
created film dict
waiting for image file
----------Run Initialized!----------


In [77]:
Process3.answer_movie_question()

Sentence with tags: 
       WORD  POS      TAG
0       Who   WP        0
1  directed  VBD        0
2       The   DT  B-TITLE
3      Lion  NNP  I-TITLE
4      King  NNP  I-TITLE
5         ?    .  I-TITLE
#################################################################
[]
['directed']
Asking for: director in: The Lion King ? or actor : 
subject: The Lion King ? | actor:  | predicate: director | predicate ID: P57
##################################################################
Not found in crowdsourcing
Not found in graph
Could not find: The Lion King


In [40]:
'''
Who is the director of Good Will Hunting?
Who is the director of Titanic?
Who produced The Lion King?


Who is the screenwriter of The Masked Gang: Cyprus
Who is the screenwriter of Titanic?
Who is the screenwriter of Weathering with You

Who is the director of Star Wars: Episode VI - Return of the Jedi?
Who is the executive producer of X-Men: First Class?
What is the genre of The Lion King?

What is the MPAA film rating of Weathering with You?
What is the MPAA film rating of Titanic?
What is the box office of The Princess and the Frog?
Can you tell me the publication date of Tom Meets Zizou?

Show me a picture of Julia Roberts.

The Lion King
The Notebook
Titanic
'''


'\nWho is the director of Good Will Hunting?\nWho is the director of Titanic?\n\nWho is the screenwriter of The Masked Gang: Cyprus\nWho is the screenwriter of Titanic?\nWho is the screenwriter of Weathering with You\n\nWho is the director of Star Wars: Episode VI - Return of the Jedi?\nWho is the executive producer of X-Men: First Class?\n\nWhat is the MPAA film rating of Weathering with You?\nWhat is the MPAA film rating of Titanic?\nWhat is the box office of The Princess and the Frog?\nCan you tell me the publication date of Tom Meets Zizou?\n\nThe Lion King\nThe Notebook\nTitanic\n'

In [98]:
def predicateID_query(proposition):
    if " " in proposition:
        proposition.replace(" ", "_")

        query ="""

        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>

        SELECT ?obj WHERE {
            ?obj rdfs:label """ + '"' + proposition + '"' + """@en .

        }
        """
        entity =moviegraph.query(query)
        print(url)
        df = pd.DataFrame(entity.bindings)
        url = str((df.iloc[0][0]))
        id = url[len(QueryProducer.WDT):]

        return id

print(predicateID_query("genre"))

None


In [68]:
def any_property(title, id, propertyname):
    if " " in propertyname:
        propertyname = propertyname.replace(" ", "_")
    WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
    WD = rdflib.Namespace("http://www.wikidata.org/entity/")

    query ='''
            SELECT ?name
            WHERE {
            ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                  rdfs:label "''' + title + '''"@en ;
                  ns1:'''+id+''' ?'''+propertyname+''' .
            ?'''+propertyname+''' rdfs:label ?name
            }
            '''
    response = moviegraph.query(query)
    response = { str(i) for i in response}

    return  response
re = any_property("Johnny English", "P179", "part of the series")
print(re)

{"(rdflib.term.Literal('Johnny English film series', lang='en'),)"}


In [ ]:
keys = pd.DataFrame(dict, index=[0])


In [ ]:
shit = "x–men: first class"

ok = "x-men: first class"

shiteee = "-"
okeier = "-"
print(shiteee == okeier)

In [ ]:
print(keys[ok])

In [ ]:
entity_emb, relation_emb, ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent = embedding

In [ ]:
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
WD = rdflib.Namespace("http://www.wikidata.org/entity/")

title = "Star Wars: Episode VI – Return of the Jedi"

query ='''
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wds: <http://www.wikidata.org/entity/statement/>
            PREFIX wdv: <http://www.wikidata.org/value/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX p: <http://www.wikidata.org/prop/>
            PREFIX ps: <http://www.wikidata.org/prop/statement/>
            PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX bd: <http://www.bigdata.com/rdf#>

            SELECT ?name
            WHERE {
            ?film wdt:P31 <http://www.wikidata.org/entity/Q11424> ;
                  rdfs:label "''' + title + '''"@en ;
                  wdt:P57 ?director .
            ?director rdfs:label ?name
            }


            '''

response = moviegraph.query(query)
response = { str(i) for i in response}
print(response)

In [ ]:
ID_to_actor = {e[len(WD):]: str(i) for e,i, IDMb in response}
actor_to_Id = {value:key for key, value in ID_to_actor.items()}

In [ ]:
WD = rdflib.Namespace("http://www.wikidata.org/entity/")

result = pd.DataFrame([
    (id[len(WD):], label, ipbm_id)
    for id, label, ipbm_id in response],
    columns=('ID', "label", 'Imdb'))

In [ ]:
s = "Lord of the Rings : the two towers"

s1 = s.replace(" :", ":")

print(s1)